In [0]:
from google.colab import drive
import gzip
drive.mount('/content/drive/')
__dir__ = '/content/drive/My Drive/Colab Notebooks/'
DIR_PATH = '/content/drive/My Drive/Colab Notebooks/data/'
GZIP_FILE = __dir__ + '/data/GoogleNews-vectors-negative300.bin.gz'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
EMBED_DIM = 300
MAX_SEQ = 26

In [0]:
import tensorflow as tf
# import tensorflow_hub as hub
import pandas as pd
import nltk
import ast
import gensim
from smart_open import open
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter, defaultdict, OrderedDict
import pandas as pd
from itertools import chain

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support,classification_report
from sklearn.preprocessing import LabelEncoder
from keras import preprocessing as prepro
from keras.layers import TimeDistributed, Dense, Activation, Input, Conv1D, Embedding, GlobalMaxPooling1D, Dropout
from keras.layers import concatenate, SimpleRNN, LSTM, MaxPooling1D
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import metrics

from tensorflow.python.tools import inspect_checkpoint as chkp
%matplotlib inline
print(tf.__version__)

1.14.0


In [0]:
print('This may take a few minutes..')
model = gensim.models.KeyedVectors.load_word2vec_format(GZIP_FILE, binary=True)
# model.save(word_embeddings)
# model.save_word2vec_format(DATA_FILE, binary=False)
print('Done!')

This may take a few minutes..


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Done!


In [0]:
def get_df(data_type):
  data_type_dict = {'train' : '/content/drive/My Drive/Colab Notebooks/data/onto.train.ner_bilou.csv',
                   'dev' : '/content/drive/My Drive/Colab Notebooks/data/onto.development.ner_bilou.csv',
                   'test' : '/content/drive/My Drive/Colab Notebooks/data/onto.test.ner_bilou.csv'}

  df = pd.read_csv(data_type_dict[data_type])
  
  X = df['x'].apply(ast.literal_eval)
  y = df['y'].apply(ast.literal_eval)
  return X, y
  
  
def split_list_on_break(df):
    df = df.fillna('<BREAK>')
    list_of_df=[]
    for columns in df.columns:
        word_seq = []
        word_sentences = []
        for count, word in enumerate(df):
            if word != '<BREAK>':
                word_seq.append(str(word))
                
            else:
                word_sentences.append(' '.join(word_seq))
                word_seq = []
                continue
        print(f'Column {columns}, total items - {count}')
        print('adding to master list..')
        list_of_df.append(word_sentences)
        
#    print(f'total tokens - {count}')
    print('returning list..')
    return list_of_df
    
    
# train_dataset = split_list_on_break(df)

In [0]:
X_train, y_train = get_df('train')
X_dev, y_dev = get_df('dev')
X_test, y_test = get_df('test')

In [0]:
def filter_data_seq(X):
  max_length = MAX_SEQ
  lengths = [len(x) for x in X]
  print(f'Maximum length: {max(lengths)}')
  print(f'Minimum length: {min(lengths)}')
  print(f'Average length: {sum(lengths)/len(lengths)}')
  
  short_sentences = [s for s in X if len(s) <= max_length]
  print(f'% of short sentences - {100 * len(short_sentences)/len(lengths)}%')

filter_data_seq(X_test)

Maximum length: 151
Minimum length: 1
Average length: 18.83588442334452
% of short sentences - 77.30211999672588%


In [0]:
print(len(X_test))

12217


In [0]:
corpus = [token for sentence in X_train for token in sentence]
vocab = ['<PAD>', '<UNK>'] + [token for token, count in Counter(corpus).items() if count >= 2]
print(f'Length of vocabulary - {len(vocab)}')
token2index = defaultdict(lambda: 1, {token: index for index, token in  enumerate(vocab)})

Length of vocabulary - 34673


In [0]:
embeddings = np.zeros((len(token2index.keys()), EMBED_DIM))
no_embed = []
print(embeddings.shape)

for k, v in token2index.items():
   try:
      embeddings[v] = model[k]
   except:
      no_embed.append((k,v))
      continue

(34673, 300)


In [0]:
print(f'% of volcabulary with no embeddings - {100 * len(no_embed)/len(vocab)}%')

% of volcabulary with no embeddings - 8.268681683154039%


In [0]:
label_vocab = ['<PAD>'] + sorted(list(set([label for target_list in y_train for label in target_list])))

label2index = {label : index for index, label in enumerate(label_vocab)}
index2label = dict([[v,k] for k,v in label2index.items()])
print(f'Length of labels - {len(label2index)}')

max_label = len(label2index)
# print(label2index)

Length of labels - 74


In [0]:
TEST_FILE = 'bios-tagged-final-flat_bilou.csv'


In [0]:
data = pd.read_csv(os.path.join(DIR_PATH, TEST_FILE), header=0, index_col=0,skip_blank_lines=False, engine='python')
data = data[data.x.str.len() > 2]

data.x = data.x.apply(ast.literal_eval)
data.y = data.y.apply(ast.literal_eval)

In [0]:
data.y.head()

0    [B-PERSON, I-PERSON, L-PERSON, O, B-DATE, L-DA...
1    [O, U-PERSON, O, O, U-TITLE, O, O, O, B-DATE, ...
2    [O, U-PERSON, O, O, O, O, O, O, O, U-TITLE, O,...
3    [B-PERSON, I-PERSON, L-PERSON, O, B-DATE, L-DA...
4    [O, U-PERSON, O, O, O, B-TITLE, I-TITLE, I-TIT...
Name: y, dtype: object

In [0]:
X_bios = data.x.tolist()
y_bios = data.y.tolist()

y_bios_no_title = [['O' if label in ['B-TITLE','U-TITLE','L-TITLE','I-TITLE'] else label for label in seq] for seq in y_bios]
print(y_bios_no_title[0])

['B-PERSON', 'I-PERSON', 'L-PERSON', 'O', 'B-DATE', 'L-DATE', 'O', 'O', 'O', 'O', 'O', 'O', 'U-ORG', 'O', 'O', 'O', 'U-ORG', 'O', 'U-DATE', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'L-DATE', 'O']


In [0]:
print(X_bios[0])
print(y_bios[0])
print(len(X_bios[500]))
print(len(y_bios[500]))

['Jeffrey', 'P.', 'Bezos', ',', 'age', '55', ',', 'has', 'been', 'Chairman', 'of', 'the', 'Board', 'since', 'founding', 'the', 'Company', 'in', '1994', 'and', 'Chief', 'Executive', 'Officer', 'since', 'May', '1996', '.']
['B-PERSON', 'I-PERSON', 'L-PERSON', 'O', 'B-DATE', 'L-DATE', 'O', 'O', 'O', 'U-TITLE', 'O', 'O', 'U-ORG', 'O', 'O', 'O', 'U-ORG', 'O', 'U-DATE', 'O', 'B-TITLE', 'I-TITLE', 'L-TITLE', 'O', 'B-DATE', 'L-DATE', 'O']
34
34


In [0]:
X_digitized = [[token2index[word] for word in seq] for seq in X_train] 
y_digitized = [[label2index[label] for label in seq] for seq in y_train]

X_dev_digitized = [[token2index[word] for word in seq] for seq in X_dev] 
y_dev_digitized = [[label2index[label] for label in seq] for seq in y_dev] 

X_test_digitized = [[token2index[word] for word in seq] for seq in X_test] 
y_test_digitized = [[label2index[label] for label in seq] for seq in y_test] 

X_bios_digitized = [[token2index[word] for word in seq] for seq in X_bios] 
y_bios_digitized = [[label2index[label] for label in seq] for seq in y_bios_no_title] 


y_digitized  = [to_categorical(target, max_label) for target in y_digitized]
y_dev_digitized  = [to_categorical(target, max_label) for target in y_dev_digitized]
y_test_digitized  = [to_categorical(target, max_label) for target in y_test_digitized]
y_bios_digitized  = [to_categorical(target, max_label) for target in y_bios_digitized]

In [0]:
X_padded = prepro.sequence.pad_sequences(X_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)
y_padded = prepro.sequence.pad_sequences(y_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)

X_dev_padded = prepro.sequence.pad_sequences(X_dev_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)
y_dev_padded = prepro.sequence.pad_sequences(y_dev_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)

X_test_padded = prepro.sequence.pad_sequences(X_dev_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)
y_test_padded = prepro.sequence.pad_sequences(y_dev_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)

X_bios_padded = prepro.sequence.pad_sequences(X_bios_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)
y_bios_padded = prepro.sequence.pad_sequences(y_bios_digitized, maxlen=MAX_SEQ, padding='post', truncating='post', value=0.0)

In [0]:
print(X_digitized[0])
print(X_padded[0].shape)
print(y_padded[0].shape)

[2, 3, 4, 5, 6]
(26,)
(26, 74)


In [0]:
VOCAB_SIZE = len(vocab)
print(MAX_SEQ)
CONV_FILTER = [5000,300,500,400,500]
CONV_KERNEL = [5,4,3,2,1]
BATCH_SIZE = 32
EPOCHS = 5
TAG_SIZE = max_label
DROP_RATE = .2

26


In [0]:
l_inp = Input(shape=(MAX_SEQ,))
l_embed = Embedding(VOCAB_SIZE, EMBED_DIM, weights=[embeddings], trainable = True, input_length=MAX_SEQ)(l_inp)


l_conv = Conv1D(CONV_FILTER[0], CONV_KERNEL[0], padding='same', activation='relu')(l_embed)
l_conv = Conv1D(CONV_FILTER[1], CONV_KERNEL[1], padding='same', activation='relu')(l_conv)
# l_conv = Conv1D(CONV_FILTER[2], CONV_KERNEL[2], padding='same', activation='relu')(l_conv)
# l_conv = Conv1D(CONV_FILTER[3], CONV_KERNEL[3], padding='same', activation='relu')(l_conv)
# l_conv = Conv1D(CONV_FILTER[4], CONV_KERNEL[4], padding='same', activation='relu')(l_conv)

l_dense1 = Dense(256, activation='relu')(l_conv)
# l_dense2 = Dense(64, activation='relu')(l_dense1)
l_output = Dense(TAG_SIZE, activation='softmax')(l_dense1)

In [0]:
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adam_customized = Adam(lr=0.0004, beta_1=0.85, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [0]:
model = Model(inputs=l_inp, outputs=l_output)
model.compile(loss='categorical_crossentropy', optimizer=adam_customized, metrics=['acc'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 26)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 26, 300)           10401900  
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 26, 5000)          7505000   
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 26, 300)           6000300   
_________________________________________________________________
dense_12 (Dense)             (None, 26, 256)           77056     
_________________________________________________________________
dense_13 (Dense)             (None, 26, 74)            19018     
Total params: 24,003,274
Trainable params: 24,003,274
Non-trainable params: 0
________________________________________________________________

In [0]:
model.reset_states()
r = model.fit(X_padded, y_padded, epochs=EPOCHS, validation_data=(X_dev_padded, y_dev_padded), batch_size=BATCH_SIZE)

Train on 115812 samples, validate on 15680 samples
Epoch 1/5
115812/115812 [==============================] - 353s 3ms/step - loss: 0.0918 - acc: 0.6019 - val_loss: 0.0825 - val_acc: 0.6160
Epoch 2/5
115812/115812 [==============================] - 349s 3ms/step - loss: 0.0460 - acc: 0.6119 - val_loss: 0.0834 - val_acc: 0.6165
Epoch 3/5
115812/115812 [==============================] - 348s 3ms/step - loss: 0.0282 - acc: 0.6170 - val_loss: 0.0989 - val_acc: 0.6157
Epoch 4/5
115812/115812 [==============================] - 347s 3ms/step - loss: 0.0164 - acc: 0.6207 - val_loss: 0.1147 - val_acc: 0.6157
Epoch 5/5
115812/115812 [==============================] - 347s 3ms/step - loss: 0.0105 - acc: 0.6228 - val_loss: 0.1431 - val_acc: 0.6155


In [0]:
def reshape_predictions(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

In [0]:

y_pred = model.predict(X_padded).argmax(2)
yh = y_padded.argmax(2)
fyh, fpr = reshape_predictions(yh, y_pred)


In [0]:
print('Training accuracy:', accuracy_score(fyh, fpr))
# print('Training confusion matrix:')
cm = confusion_matrix(fyh, fpr)

Training accuracy: 0.6243464208571451


In [0]:
print(classification_report(fyh, fpr))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00   1125577
           1       0.98      0.98      0.98      1853
           2       0.97      0.97      0.97      9847
           3       0.96      0.99      0.98       695
           4       0.99      0.98      0.98       762
           5       0.98      0.98      0.98      3237
           6       1.00      0.89      0.94         9
           7       0.97      0.98      0.98       305
           8       1.00      0.92      0.95       954
           9       0.99      0.99      0.99      3536
          10       1.00      0.97      0.98       417
          11       0.71      0.62      0.67         8
          12       0.99      0.99      0.99     11897
          13       0.98      0.98      0.98      2995
          14       1.00      0.99      1.00      9713
          15       0.96      0.98      0.97       327
          16       0.96      0.99      0.98       799
          17       0.98    

In [0]:
y_pred_test = model.predict(X_test_padded).argmax(2)
yh_test = y_test_padded.argmax(2)
fyh_test, fpr_test = reshape_predictions(yh_test, y_pred_test)
print('Training accuracy:', accuracy_score(fyh_test, fpr_test))

Training accuracy: 0.6154974489795918


In [0]:
print(classification_report(fyh_test, fpr_test))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    146943
           1       0.70      0.66      0.68       312
           2       0.82      0.75      0.78      1539
           3       0.57      0.54      0.55       113
           4       0.53      0.40      0.46        97
           5       0.83      0.80      0.82       583
           6       0.00      0.00      0.00         1
           7       0.71      0.57      0.63        44
           8       0.79      0.37      0.50       149
           9       0.89      0.89      0.89       521
          10       0.83      0.49      0.62        51
          11       0.00      0.00      0.00         2
          12       0.85      0.81      0.83      1850
          13       0.89      0.92      0.91       556
          14       0.90      0.85      0.88      1428
          15       0.49      0.26      0.34        89
          16       0.77      0.69      0.73       141
          17       0.67    

In [0]:
y_pred_bios = model.predict(X_bios_padded).argmax(2)
yh_bios = y_bios_padded.argmax(2)
fyh_bios, fpr_bios = reshape_predictions(yh_bios, y_pred_bios)
print('Training accuracy:', accuracy_score(fyh_bios, fpr_bios))

Training accuracy: 0.5680682312038244


In [0]:
print('Training confusion matrix:')
print(confusion_matrix(fyh_bios, fpr_bios))

Training confusion matrix:
[[  0   0   0 ...   0   0   0]
 [  0 125   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ... 188   0   0]
 [  0   0   0 ...   2   0   0]
 [  0   0   0 ...   0   0   0]]


In [0]:
y_bios_flat = chain.from_iterable(data.y.tolist())


counts = Counter(y_bios_flat)

print(counts)

Counter({'O': 10059, 'I-ORG': 1278, 'B-ORG': 886, 'L-ORG': 886, 'U-TITLE': 498, 'B-DATE': 340, 'L-DATE': 340, 'B-TITLE': 311, 'L-TITLE': 311, 'I-DATE': 309, 'I-TITLE': 275, 'U-PERSON': 252, 'U-ORG': 239, 'U-DATE': 227, 'B-PERSON': 102, 'L-PERSON': 102, 'I-PERSON': 77, 'U-GPE': 47, 'I-PRODUCT': 27, 'U-PRODUCT': 27, 'B-GPE': 22, 'L-GPE': 22, 'B-PRODUCT': 17, 'L-PRODUCT': 17, 'I-GPE': 7, 'U-NORP': 4, 'B-MONEY': 3, 'L-MONEY': 3, 'B-LAW': 3, 'L-LAW': 3, 'B-PERCENT': 2, 'L-PERCENT': 2, 'U-CARDINAL': 2, 'I-LAW': 2, 'B-NORP': 2, 'L-NORP': 2, 'U-ORDINAL': 2, 'U-LOC': 2, 'I-MONEY': 2})


In [0]:
# OrderedDict(counts)

In [0]:
print(classification_report(fyh_bios, fpr_bios))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5817
           2       0.76      0.51      0.61       246
           3       0.00      0.00      0.00         0
           5       0.50      0.12      0.19        17
           7       0.50      0.33      0.40         3
           8       0.00      0.00      0.00         0
           9       1.00      1.00      1.00         1
          10       0.00      0.00      0.00         2
          12       0.72      0.61      0.66       699
          13       0.00      0.00      0.00         1
          14       0.84      0.81      0.83        97
          15       0.00      0.00      0.00        15
          18       0.00      0.00      0.00         0
          20       0.63      0.42      0.51       177
          22       0.00      0.00      0.00         0
          23       0.50      0.40      0.44         5
          25       0.06      0.50      0.11         2
          26       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
score = model.evaluate(X_bios_padded, y_bios_padded)
print(f'Model loss: {score}')

708/708 [==============================] - 1s 822us/step
Model loss: [0.8057204478204587, 0.5680682335869741]
